In [5]:
#!pip install sklearn

In [6]:
from sklearn.datasets import load_files
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch
from sklearn.decomposition import TruncatedSVD
import json

In [7]:
with open("C:\\Users\\tuyen.dv\\Documents\\NLP\\NLP_tuan_3\\Data\\datasetVNTC.json", encoding="utf-8") as f:
  data_train = json.load(f)

In [8]:
data_train['target']

[3,
 8,
 3,
 8,
 4,
 5,
 6,
 6,
 6,
 5,
 2,
 0,
 6,
 0,
 1,
 9,
 4,
 0,
 3,
 3,
 4,
 0,
 6,
 0,
 8,
 6,
 4,
 0,
 7,
 3,
 7,
 0,
 5,
 5,
 8,
 9,
 1,
 6,
 3,
 3,
 2,
 0,
 9,
 9,
 5,
 6,
 6,
 8,
 5,
 7,
 3,
 3,
 5,
 5,
 6,
 0,
 5,
 4,
 8,
 9,
 1,
 9,
 6,
 2,
 2,
 3,
 4,
 0,
 3,
 0,
 2,
 9,
 3,
 4,
 3,
 4,
 7,
 0,
 3,
 6,
 9,
 0,
 6,
 3,
 8,
 5,
 2,
 0,
 0,
 9,
 0,
 5,
 0,
 7,
 7,
 3,
 0,
 8,
 6,
 7,
 6,
 4,
 0,
 1,
 6,
 4,
 3,
 5,
 3,
 1,
 0,
 0,
 9,
 6,
 6,
 9,
 9,
 8,
 6,
 9,
 9,
 9,
 8,
 7,
 2,
 5,
 0,
 0,
 5,
 2,
 0,
 0,
 0,
 7,
 5,
 6,
 6,
 9,
 8,
 8,
 8,
 0,
 4,
 6,
 6,
 8,
 2,
 9,
 4,
 4,
 6,
 7,
 9,
 1,
 3,
 5,
 0,
 4,
 6,
 6,
 1,
 6,
 8,
 0,
 2,
 4,
 4,
 9,
 1,
 9,
 1,
 6,
 2,
 0,
 5,
 4,
 6,
 8,
 7,
 3,
 4,
 6,
 6,
 0,
 8,
 3,
 3,
 9,
 7,
 7,
 9,
 4,
 7,
 0,
 3,
 3,
 2,
 3,
 9,
 3,
 4,
 3,
 3,
 0,
 3,
 8,
 6,
 0,
 0,
 7,
 9,
 6,
 8,
 5,
 2,
 5,
 1,
 7,
 2,
 9,
 6,
 6,
 3,
 8,
 8,
 9,
 6,
 6,
 3,
 6,
 4,
 0,
 1,
 1,
 6,
 6,
 7,
 5,
 6,
 4,
 0,
 8,
 8,
 5,
 6,
 0,
 6,
 2,
 2,
 4,


In [9]:
with open("C:\\Users\\tuyen.dv\\Documents\\NLP\\NLP_tuan_3\\Data\\stopword.txt", encoding="utf-8") as f:
  stopwords = f.readlines()
stopwords = [x.strip().replace(" ", "_") for x in stopwords]

In [10]:
tfidf_vector = TfidfVectorizer(stop_words=stopwords)

In [11]:
tfidf_matrix = tfidf_vector.fit_transform(data_train['data'], data_train['target'])

In [12]:
svd = TruncatedSVD(n_components=300)
svd.fit(tfidf_matrix)
svd_tfidf_vector = svd.transform(tfidf_matrix)

In [13]:
print(svd_tfidf_vector.shape)

(33759, 300)


In [14]:
test_size = 0.2
X_train, X_test, y_train, y_test = train_test_split( svd_tfidf_vector,
data_train['target'], test_size=test_size)

In [15]:
import torch
from torch import nn
import torch.optim as optim

In [16]:
class MultilayerPerceptron(nn.Module):

  def __init__(self, input_size,output_size):
    super(MultilayerPerceptron, self).__init__()

    # Saving the initialization parameters
    self.input_size = input_size 
    self.output_size = output_size

    # Defining model
    self.model = nn.Sequential(
        nn.Linear(self.input_size, 128),
        nn.ReLU(),
        nn.Linear(128, 32),
        nn.ReLU(),
        nn.Linear(32, output_size),
    )
    
  def forward(self, x):
    output = self.model(x)
    return output

In [17]:
# Instantiate the model
model = MultilayerPerceptron(X_train.shape[1], 10)

# Define the optimizer
adam = optim.Adam(model.parameters(), lr=1e-1, weight_decay=0.001)

# Define loss using a predefined loss function
loss_function = nn.CrossEntropyLoss()
data_test = data_train
# # Calculate how our model is doing now
# y_pred = model(x)
# loss_function(y_pred, y).item()

In [18]:
Y_train = torch.tensor(y_train, dtype= torch.long)

In [19]:
n_epoch = 100
for epoch in range(n_epoch):
  # Set the gradients to 0
  adam.zero_grad()
  # Get the model predictions
  y_pred = model(torch.tensor(X_train, dtype = torch.float))
  # Get the loss
  loss = loss_function(y_pred, Y_train)
  # Print stats

  print(f"Epoch {epoch}: traing loss: {loss}")

  # Compute the gradients
  loss.backward()
  # Take a step to optimize the weights
  adam.step()

Epoch 0: traing loss: 2.3264851570129395
Epoch 1: traing loss: 2.2802205085754395
Epoch 2: traing loss: 2.3252296447753906
Epoch 3: traing loss: 2.2210195064544678
Epoch 4: traing loss: 2.159334659576416
Epoch 5: traing loss: 2.0921943187713623
Epoch 6: traing loss: 1.958629846572876
Epoch 7: traing loss: 1.8647695779800415
Epoch 8: traing loss: 1.8193422555923462
Epoch 9: traing loss: 1.7408756017684937
Epoch 10: traing loss: 1.6251813173294067
Epoch 11: traing loss: 1.5285511016845703
Epoch 12: traing loss: 1.3639805316925049
Epoch 13: traing loss: 1.2543139457702637
Epoch 14: traing loss: 1.1352627277374268
Epoch 15: traing loss: 1.0166913270950317
Epoch 16: traing loss: 0.9454199075698853
Epoch 17: traing loss: 0.928065836429596
Epoch 18: traing loss: 0.9254903197288513
Epoch 19: traing loss: 0.7861998677253723
Epoch 20: traing loss: 0.7897370457649231
Epoch 21: traing loss: 0.6490819454193115
Epoch 22: traing loss: 0.6370174884796143
Epoch 23: traing loss: 0.5421664118766785
Epoch

In [20]:
def evaluate(X_test, y_test, model_func):
  pred = model_func(torch.tensor(X_test, dtype = torch.float))
  output = np.argmax(pred.tolist(), axis=1)
  count = 0
  n = X_test.shape[0]
  for i in range(n):
    if(output[i] == y_test[i]):
      count += 1
  return float(count)*100/n

In [22]:
print(X_train.shape)

(27007, 300)


In [21]:
print(evaluate(X_train, y_train, model))
print(evaluate(X_test, y_test, model))

91.89469396823046
90.16587677725119


In [23]:
#save model
torch.save(model.state_dict(), "ann_tfidf.pth")

In [24]:
#load model
model_1 = MultilayerPerceptron(X_train.shape[1], 10)
model_1.load_state_dict(torch.load('ann_tfidf.pth'))

<All keys matched successfully>

In [25]:
print(evaluate(X_train, y_train, model_1))
print(evaluate(X_test, y_test, model_1))

91.89469396823046
90.16587677725119
